In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs

In [10]:
df_mar = pd.read_csv("../data/Raw/Marine_Microplastics_WGS84_8553846406879449657.csv")
df_mar

,OBJECTID,Latitude (degree),Longitude(degree),Ocean,Region,Subregion,Country,State,Beach Location,Marine Setting,...,DOI,ORGANIZATION,KEYWORDS,NCEI Accession No,NCEI Accession No. Link,Symbology,GlobalID,Date (MM-DD-YYYY),x,y
0,1,45.280000,-60.290000,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",2ce1d1d4-4231-4ab8-af74-0b7fbb1e028b,7/13/1989 12:00:00 AM,-60.290000,45.280000
1,2,40.930000,-70.650000,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Low, 0.0005-0.005 pieces/m3",8e3c8426-5ab0-421e-abb2-6b90197ada7a,10/12/1989 12:00:00 AM,-70.650000,40.930000
2,3,40.930000,-70.650000,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Low, 0.0005-0.005 pieces/m3",99bda337-9f0c-4f58-9960-8b00c1110a7c,10/12/1989 12:00:00 AM,-70.650000,40.930000
3,4,40.300000,-69.770000,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",1f20e926-2a84-4e42-8478-8a5af5aecacc,10/12/1989 12:00:00 AM,-69.770000,40.300000
4,5,39.880000,-67.150000,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",a0df6bee-9c60-4621-a0bc-3329e3f11720,10/15/1989 12:00:00 AM,-67.150000,39.880000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22525,22941,-5.933333,39.360000,Indian Ocean,NaN,NaN,Tanzania,Zanzibar,Pwani-Mchangani,Ocean sediment,...,https://doi.org/10.1016/j.marpolbul.2022.114305,"University of Dar es Salaam, Tanzania",Tanzania,297601,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Sediment, High, 150-200 pieces kg-1 d.w.",47556e88-2992-48a6-a94d-97dee9ec0f4a,12/23/2019 12:00:00 AM,39.360000,-5.933333
22526,22942,-5.500000,39.120000,Indian Ocean,NaN,NaN,Tanzania,Zanzibar,Nungwi,Ocean sediment,...,https://doi.org/10.1016/j.marpolbul.2022.114305,"University of Dar es Salaam, Tanzania",Tanzania,297601,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Sediment, High, 150-200 pieces kg-1 d.w.",55430f26-f5df-44d6-86bb-a532cca54bbc,12/22/2019 12:00:00 AM,39.120000,-5.500000
22527,22943,-6.450000,39.466667,Indian Ocean,NaN,NaN,Tanzania,Zanzibar,Kizimkazi,Ocean sediment,...,https://doi.org/10.1016/j.marpolbul.2022.114305,"University of Dar es Salaam, Tanzania",Tanzania,297601,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Sediment, Medium, 20-150 pieces kg-1 d.w.",c39e2f1f-1d4b-4f2f-830d-60abe0b88671,12/25/2019 12:00:00 AM,39.466667,-6.450000
22528,22944,-6.320000,39.210000,Indian Ocean,NaN,NaN,Tanzania,Zanzibar,Bububu,Ocean sediment,...,https://doi.org/10.1016/j.marpolbul.2022.114305,"University of Dar es Salaam, Tanzania",Tanzania,297601,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Sediment, Very High, >200 pieces kg-1 d.w.",c46c197f-38df-4f28-9118-b20252f3d123,12/28/2019 12:00:00 AM,39.210000,-6.320000


In [11]:
df_mar.shape

(22530, 36)

In [12]:
df_mar.duplicated().sum()

np.int64(0)

In [13]:
for col in df_mar.columns:
    print(col)

OBJECTID
Latitude (degree)
Longitude(degree)
Ocean
Region
Subregion
Country
State
Beach Location
Marine Setting
Ocean Bottom Depth (m)
Water Sample Depth (m)
Sediment Sample Depth (m)
Sampling Method
Mesh size (mm)
Transect No
Sampling point on beach
Volunteers Number
Standardized Nurdle  Amount
Microplastics measurement
Unit
Concentration class range
Concentration class text
Short Reference
Long Reference
DOI
ORGANIZATION
KEYWORDS
NCEI Accession No
NCEI Accession No. Link
Symbology
GlobalID
Date (MM-DD-YYYY)
x
y


In [14]:
df_mar = df_mar[[
    "Microplastics measurement",
    "Unit",
    "Concentration class range",
    "Mesh size (mm)",
    "Latitude (degree)",
    "Longitude(degree)"
]]

df_mar = df_mar.rename(columns={
    "Latitude (degree)": "lat",
    "Longitude(degree)": "lon",
    "Microplastics measurement": "microplastics_measurement",
    "Concentration class range": "concentration_class_range",
    "Mesh size (mm)": "mesh_size_mm"
})

df_mar.columns = df_mar.columns.str.lower()

# Asegurar coordenadas como float
df_mar["lon"] = df_mar["lon"].astype(float)
df_mar["lat"] = df_mar["lat"].astype(float)

df_mar.head(10)

,microplastics_measurement,unit,concentration_class_range,mesh_size_mm,lat,lon
0,0.000000,pieces/m3,0-0.0005,0.335,45.28,-60.29
1,0.002276,pieces/m3,0.0005-0.005,0.335,40.93,-70.65
2,0.004320,pieces/m3,0.0005-0.005,0.335,40.93,-70.65
3,0.000000,pieces/m3,0-0.0005,0.335,40.30,-69.77
4,0.000000,pieces/m3,0-0.0005,0.335,39.88,-67.15
5,0.043196,pieces/m3,0.005-1,0.335,37.83,-63.73
6,0.535636,pieces/m3,0.005-1,0.335,37.83,-63.73
7,0.069116,pieces/m3,0.005-1,0.335,36.90,-63.38
8,0.084232,pieces/m3,0.005-1,0.335,36.90,-63.38
9,0.299496,pieces/m3,0.005-1,0.335,35.70,-61.98


In [21]:
df_mar.duplicated().sum()

np.int64(3208)

In [23]:
dups = df_mar[df_mar.duplicated()]
dups.head(10)

,microplastics_measurement,unit,concentration_class_range,mesh_size_mm,lat,lon
18,0.00000,pieces/m3,0-0.0005,0.335,17.80,-64.32
34,0.00000,pieces/m3,0-0.0005,0.335,15.10,-61.98
36,0.00000,pieces/m3,0-0.0005,0.335,13.98,-61.70
43,0.00000,pieces/m3,0-0.0005,0.335,13.05,-63.12
47,0.00000,pieces/m3,0-0.0005,0.335,14.68,-64.43
49,0.00000,pieces/m3,0-0.0005,0.335,15.80,-65.30
51,0.00432,pieces/m3,0.0005-0.005,0.335,16.48,-65.90
57,0.00000,pieces/m3,0-0.0005,0.335,17.90,-67.62
59,0.00000,pieces/m3,0-0.0005,0.335,17.90,-67.82
61,0.00000,pieces/m3,0-0.0005,0.335,18.03,-68.42


In [28]:
df_mar = df_mar.drop_duplicates()
df_mar.shape

(19322, 6)

In [29]:
df_mar = df_mar.dropna(subset=["lat", "lon"])

In [30]:
df_mar.shape

(19322, 6)

In [32]:
df_mar.head(10)

,microplastics_measurement,unit,concentration_class_range,mesh_size_mm,lat,lon
0,0.000000,pieces/m3,0-0.0005,0.335,45.28,-60.29
1,0.002276,pieces/m3,0.0005-0.005,0.335,40.93,-70.65
2,0.004320,pieces/m3,0.0005-0.005,0.335,40.93,-70.65
3,0.000000,pieces/m3,0-0.0005,0.335,40.30,-69.77
4,0.000000,pieces/m3,0-0.0005,0.335,39.88,-67.15
5,0.043196,pieces/m3,0.005-1,0.335,37.83,-63.73
6,0.535636,pieces/m3,0.005-1,0.335,37.83,-63.73
7,0.069116,pieces/m3,0.005-1,0.335,36.90,-63.38
8,0.084232,pieces/m3,0.005-1,0.335,36.90,-63.38
9,0.299496,pieces/m3,0.005-1,0.335,35.70,-61.98


In [33]:
df_mar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19322 entries, 0 to 22529
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   microplastics_measurement  14197 non-null  float64
 1   unit                       19322 non-null  object 
 2   concentration_class_range  19322 non-null  object 
 3   mesh_size_mm               14155 non-null  float64
 4   lat                        19322 non-null  float64
 5   lon                        19322 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.0+ MB


In [34]:
df_mar.to_csv("../data/GeoDataFrame/dataframe/df_mar.csv", index = False)

#### GeoDataFrame

In [37]:
gdf_mar = gpd.GeoDataFrame(
    df_mar,
    geometry=gpd.points_from_xy(df_mar.lon, df_mar.lat),
    crs="EPSG:4326"
)

In [38]:
# coordenadas fuera de rango
gdf_mar = gdf_mar[
    gdf_mar.lon.between(-180, 180) &
    gdf_mar.lat.between(-90, 90)
]

# eliminar geometrías inválidas
gdf_mar = gdf_mar[gdf_mar.is_valid]

In [39]:
gdf_mar.shape

(19322, 7)

In [40]:
gdf_mar.to_file("../data/GeoDataFrame/gdf_microplastics.gpkg", driver="GPKG")